In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import random
from sklearn.metrics import precision_recall_fscore_support as prfs
from sklearn.linear_model import SGDClassifier as LC
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier as AB
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.linear_model import LogisticRegression as LR
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.naive_bayes import GaussianNB as NB
from sklearn.ensemble import ExtraTreesClassifier as ET
from sklearn.neural_network import MLPClassifier
from nltk import ngrams
from sklearn.model_selection import GridSearchCV

In [2]:
training_set = pd.DataFrame.from_csv('/Users/accy/Dropbox/IMFS-Summer-Kaggle/dataset/Phase 1 - similarityCompetition/train.csv')
test_set = pd.DataFrame.from_csv('/Users/accy/Dropbox/IMFS-Summer-Kaggle/dataset/Phase 1 - similarityCompetition/test.csv')

# This is how training dataset looks like

In [3]:
training_set.head(3)

,description_x,description_y,ticker_x,ticker_y,same_security
test_id,,,,,
0,first trust dow jones internet,first trust dj internet idx,FDN,FDN,True
1,schwab intl large company index etf,schwab strategic tr fundamental intl large co ...,FNDF,FNDF,True
2,vanguard small cap index adm,vanguard small-cap index fund inst,VSMAX,VSCIX,False


In [4]:
training_set['same_security'].value_counts()

True     1613
False     529
Name: same_security, dtype: int64

In [5]:
training_set['same_security'].value_counts()[True]/sum(training_set['same_security'].value_counts())

0.7530345471521942

In [6]:
not_same_train_sample = training_set[training_set['same_security'] == False]
same_train_sample = training_set[training_set['same_security']]


In [7]:
len(training_set)

2142

In [8]:
training_set[['r6' in training_set['description_x'][i] for i in range(len(training_set))]]

,description_x,description_y,ticker_x,ticker_y,same_security
test_id,,,,,
81,aqr managed futures strategy r6,aqr mgd futures strat fd cl i,AQMRX,AQMIX,False
146,jh dscpl val mdcp r6,jh disciplnd val md cp r6,JVMRX,JVMRX,True
206,american funds inc fund of amer r6,american funds income fund of america r6,RIDGX,RIDGX,True
246,deut real estate r6,deutsche real estate sec a,RRRZX,RRRAX,False
257,aqr managed futures strategy r6,aqr managed futures strategy fund class i,AQMRX,AQMIX,False
350,american funds euro pacific growth r6,american europacific growth r6 m5585,RERGX,RERGX,True
458,jpmorgan core bond fund r6,jpm core bond r6,JCBUX,JCBUX,True
514,aqr managed futures strategy r6,aqr fds aqmixmanaged futures strategy fd i,AQMRX,AQMIX,False
639,af bond fd amer r6,american funds bond fund of america a,RBFGX,ABNDX,False


In [9]:
not_same_train_sample.to_csv('false_same_security.csv')

In [10]:
def is_abbr(w1, w2):
    #the function to determine whether w1 is w2's abbr or w2 is w1's abbr.
    if len(w2) > len(w1):
        temp = w1
        w1 = w2
        w2 = temp
    #make sure w1 is the longest
    w2_index, w1_index = 0, 0
    while w1_index < len(w1) and w2_index < len(w2):
        if w1[w1_index] == w2[w2_index]:
            w2_index += 1
        w1_index += 1
    return True if  w2_index == len(w2) else False
    

In [11]:
#Replace some abbrs to be their original word
#So the vector can represent the data better

rep_dic = {}

rep_dic['div:'] = ''
rep_dic['dj'] = 'dow jones'
rep_dic['&'] = 'and'
rep_dic['administr'] = 'admin'
rep_dic['adm'] = 'admin'
rep_dic['cl'] = 'class' 
rep_dic['corp'] = 'corpor'
rep_dic['hldgs'] = 'hold'
rep_dic['hldg'] = 'hold'
rep_dic['idx'] = 'index'
rep_dic['mkt'] = 'market'
rep_dic['mkts'] = 'market'
rep_dic['shrs'] = 'share'
rep_dic['shs'] = 'share'
rep_dic['stk'] = 'stock'
rep_dic['invmt'] = 'investment'
rep_dic['intl'] = 'international'
rep_dic['cl'] = 'class'
rep_dic['ltd'] = 'limited'


###############################
rep_dic['indx'] = 'index'
""" 
rep_dic['co'] = 'company'
rep_dic['com'] = 'company'
rep_dic['blkrk'] = 'blackrock'
rep_dic['hi'] = 'high'
rep_dic['hodg'] = 'hold'
rep_dic['inv'] = 'investor'

rep_dic['cls']= 'class'
rep_dic['extd'] = 'extend'
rep_dic['vang'] = 'vanguard'
rep_dic['vangua'] = 'vanguard'
rep_dic['van'] = 'vanguard'
rep_dic['yld'] = 'yield'
"""

" \nrep_dic['co'] = 'company'\nrep_dic['com'] = 'company'\nrep_dic['blkrk'] = 'blackrock'\nrep_dic['hi'] = 'high'\nrep_dic['hodg'] = 'hold'\nrep_dic['inv'] = 'investor'\n\nrep_dic['cls']= 'class'\nrep_dic['extd'] = 'extend'\nrep_dic['vang'] = 'vanguard'\nrep_dic['vangua'] = 'vanguard'\nrep_dic['van'] = 'vanguard'\nrep_dic['yld'] = 'yield'\n"

In [12]:
#transform every words to be its stem, for example: admiral -> admir
sno = nltk.stem.SnowballStemmer('english')

In [13]:
def difference(data):
    # For each data, find the difference between two descriptions
    dif_list = []
    for i in range(len(data)):
        x, y = data['description_x'][i], data['description_y'][i]
        regex = re.compile('[^a-zA-Z]')
        
            
        x = regex.sub(' ', x)
        y = regex.sub(' ', y)
        x_list, y_list = x.split(), y.split()
        for i, w in enumerate(x_list):
            x_list[i] = w
            
            if x_list[i] in rep_dic:
                x_list[i] = rep_dic[x_list[i]]
            
        for i, w in enumerate(y_list):
            y_list[i] = w
            
            if y_list[i] in rep_dic:
                y_list[i] = rep_dic[y_list[i]]
        
        s_x, s_y = set(x_list), set(y_list)
        nexact_x, nexact_y = [], []
        for w in x_list:
            if w not in s_y:
                nexact_x.append(w)
        for w in y_list:
            if w not in s_x:
                nexact_y.append(w)
        dif_x, dif_y = [], []
        for w1 in nexact_x:
            abbr = False
            for w2 in nexact_y:
                abbr = is_abbr(w1, w2)
                if abbr:
                    break
            if not abbr:
                dif_x.append(sno.stem(w1))
        
        for w2 in nexact_y:
            abbr = False
            for w1 in nexact_x:
                if min(len(w1), len(w2)) == 1:
                    continue
                abbr = is_abbr(w1, w2)
                if abbr:
                    break
            if not abbr:
                dif_y.append(sno.stem(w2))
        if '' in dif_x:
            dif_x.remove('')
        if '' in dif_y:
            dif_y.remove('')
        dif_list.append((dif_x, dif_y))
    return pd.DataFrame(data = dif_list, columns = ['x', 'y'])
    


In [14]:
dif_df = difference(training_set)
dif_df['label'] = training_set['same_security']
test_dif_df = difference(test_set)

In [15]:
ratio = 0.8
index = [i for i in range(len(dif_df))]
random.shuffle(index)
train_index, valid_index = index[:int(len(dif_df)*ratio)], index[int(len(dif_df)*ratio):]

In [16]:
dif_df.to_csv('preprocessed_data.csv')

In [17]:
train_dif_df = dif_df[[True if i in train_index else False for i in range(len(index))]]
valid_dif_df = dif_df[[True if i not in train_index else False for i in range(len(index))]]

In [18]:
train_dif_df.reset_index(inplace = True)
valid_dif_df.reset_index(inplace = True)

In [19]:
s = dict()
for i in range(len(train_dif_df)):
    for w in train_dif_df['x'][i] + train_dif_df['y'][i]:
        s[w] = s.get(w, 0) + 1
features = sorted([ k for k, v in s.items() if v >15 and len(k)>1])
feature_index = {features[i] : i for i in range(len(features))}

In [20]:
def training_sample(dif_df, feature_index):
    res_list = []
    for i in range(len(dif_df)):
        feature_list = [0] * len(feature_index.keys())
        for w in dif_df['x'][i] + dif_df['y'][i]:
            if w in feature_index:
                feature_list[feature_index[w]] = 1
        feature_list.append(len(dif_df['x'][i] + dif_df['y'][i]) - sum(feature_list))
        feature_list.append(len(dif_df['x'][i] + dif_df['y'][i]))
        res_list.append(feature_list)
    return pd.DataFrame(data = res_list, columns = list(feature_index.keys())+['Number of Unknown', 'Number of Dif'])



In [21]:
X, y = training_sample(train_dif_df, feature_index), train_dif_df['label']
valid_X, valid_y = training_sample(valid_dif_df, feature_index), valid_dif_df['label']
test_X = training_sample(test_dif_df, feature_index)


# Baseline

In [22]:
valid_y.value_counts(True)

True     0.7669
False    0.2331
Name: label, dtype: float64

In [23]:
X.to_csv('training_X.csv')

# SVM

In [24]:
svc = SVC(C=100)
svc.fit(X, y)
prediction = svc.predict(valid_X)
res = prediction == valid_y
pre, recall, f1, sth = prfs(valid_y, prediction, average = 'binary')
acc = res.value_counts()[True]/len(res)
print('Precision: ', round(pre,3), ' Recall: ', round(recall,3), ' F1: ', round(f1,3), ' Accuracy: ', round(acc,3))



Precision:  0.917  Recall:  0.97  F1:  0.942  Accuracy:  0.909


In [25]:
mis_index = list(valid_dif_df[prediction != valid_y]['index'])
mis_df = pd.DataFrame()
mis_df['decription_x'] = training_set['description_x'][mis_index]
mis_df['decription_y'] = training_set['description_y'][mis_index]
mis_df['same_security'] = training_set['same_security'][mis_index]
mis_df.to_csv('data_misclassified_by_SVM.csv')

# AdaBoost

In [26]:
ab = AB()
ab.fit(X, y)
prediction = ab.predict(valid_X)
res = prediction == valid_y
pre, recall, f1, sth = prfs(valid_y, prediction, average = 'binary')
acc = res.value_counts()[True]/len(res)
print('Precision: ', round(pre,3), ' Recall: ', round(recall,3), ' F1: ', round(f1,3), ' Accuracy: ', round(acc,3))




Precision:  0.915  Recall:  0.951  F1:  0.933  Accuracy:  0.895


# Random Forest

In [27]:
rf = RF(20)
rf.fit(X, y)
prediction = rf.predict(valid_X)
res = prediction == valid_y
pre, recall, f1, sth = prfs(valid_y, prediction, average = 'binary')
acc = res.value_counts()[True]/len(res)
print('Precision: ', round(pre,3), ' Recall: ', round(recall,3), ' F1: ', round(f1,3), ' Accuracy: ', round(acc,3))

Precision:  0.924  Recall:  0.957  F1:  0.94  Accuracy:  0.907


In [28]:
lr = LR()
lr.fit(X, y)
prediction = lr.predict(valid_X)
res = prediction == valid_y
pre, recall, f1, sth = prfs(valid_y, prediction, average = 'binary')
acc = res.value_counts()[True]/len(res)
print('Precision: ', round(pre,3), ' Recall: ', round(recall,3), ' F1: ', round(f1,3), ' Accuracy: ', round(acc,3))

Precision:  0.892  Recall:  0.954  F1:  0.922  Accuracy:  0.876


In [29]:
knn = KNN()
knn.fit(X, y)
prediction = knn.predict(valid_X)
res = prediction == valid_y
pre, recall, f1, sth = prfs(valid_y, prediction, average = 'binary')
acc = res.value_counts()[True]/len(res)
print('Precision: ', round(pre,3), ' Recall: ', round(recall,3), ' F1: ', round(f1,3), ' Accuracy: ', round(acc,3))

Precision:  0.909  Recall:  0.942  F1:  0.925  Accuracy:  0.883


In [30]:
nb = NB()
nb.fit(X, y)
prediction = nb.predict(valid_X)
res = prediction == valid_y
pre, recall, f1, sth = prfs(valid_y, prediction, average = 'binary')
acc = res.value_counts()[True]/len(res)
print('Precision: ', round(pre,3), ' Recall: ', round(recall,3), ' F1: ', round(f1,3), ' Accuracy: ', round(acc,3))

Precision:  0.931  Recall:  0.328  F1:  0.485  Accuracy:  0.466


In [31]:
et = ET(20)
et.fit(X, y)
prediction = et.predict(valid_X)
res = prediction == valid_y
pre, recall, f1, sth = prfs(valid_y, prediction, average = 'binary')
acc = res.value_counts()[True]/len(res)
print('Precision: ', round(pre,3), ' Recall: ', round(recall,3), ' F1: ', round(f1,3), ' Accuracy: ', round(acc,3))

Precision:  0.929  Recall:  0.951  F1:  0.94  Accuracy:  0.907


In [32]:
mis_index = list(valid_dif_df[prediction != valid_y]['index'])
mis_df = pd.DataFrame()
mis_df['decription_x'] = training_set['description_x'][mis_index]
mis_df['decription_y'] = training_set['description_y'][mis_index]
mis_df['same_security'] = training_set['same_security'][mis_index]
mis_df.to_csv('data_misclassified_by_ExtraTree.csv')

In [33]:
mis_df

,decription_x,decription_y,same_security
test_id,,,
87,"energy transfer partners, l.p. cmn",cheniere energy partners lp com,False
132,vanguard mid-cap index fund admiral shares,van mid cap index adm m4940,True
143,dfa us large cap value prtf instl,dfa us large cap value,True
172,fidelity focused stock fund,fidelity small cap growth fund,False
223,growth fund of america class a - american fund...,american growth fund of america class a,True
256,invesco growth & income fund cl y,invesco growth and income y,True
276,fid eq div income k,fidelity equity dividend income fund,False
304,hodges small cap retail,fidelity small cap stock,False
387,royce value trust inc div: 1.200,keycorp new div: 0.300,False


In [34]:
valid_dif_df[prediction != valid_y]

,index,x,y,label
19,87,"[transfer, cmn]","[chenier, lp, com]",False
27,132,"[fund, share]","[admin, m]",True
29,143,"[prtf, instl]",[],True
30,172,"[focus, stock]","[small, cap, growth]",False
40,223,"[fund, m, f]",[],True
45,256,"[fund, class]",[and],True
50,276,[k],[fund],False
56,304,"[hodg, retail]","[fidel, stock]",False
65,387,"[royc, valu, trust, inc]","[keycorp, new]",False
77,429,[],"[eqti, port, instl]",True


In [35]:
train_X = pd.concat([X, valid_X])
train_y = pd.concat([y, valid_y])

In [36]:
et = ET(20)
et.fit(train_X, train_y)
prediction = et.predict(test_X)
test_set['same_security'] = prediction
test_set.to_csv(path_or_buf = '/Users/accy/Documents/GitHub/Vanguard_Kaggle/submission/Phase 1 - submission/submission_phase1_team1.csv', index = False)

In [37]:
from pyxdameraulevenshtein import damerau_levenshtein_distance as dld

In [38]:
def combine_single_char(s):
    #deal with single char like ' u s ' and ' u s a '
    index = 0
    res = []
    while index < len(s):
        if len(s[index]) == 1:
            start_index = index
            while index + 1 < len(s) and len(s[index+1]) == 1:
                index += 1
            res.append(''.join(s[start_index:index + 1]))
        else:
            res.append(s[index])
        index += 1
    return res
        

In [39]:
def difference2(data):
    # For each data, find the difference between two descriptions
    dif_list = []
    pre_determine = []
    for i in range(len(data)):
        x, y = data['description_x'][i], data['description_y'][i]
        div = None
        if 'div:' in x and 'div:' in y:
            div = False
        elif 'div:' in x or 'div:' in y:
            div = True    
        regex = re.compile('[^a-zA-Z]')
        x = regex.sub(' ', x)
        y = regex.sub(' ', y)
        x_list, y_list = x.split(), y.split()
        x_list, y_list = combine_single_char(x_list), combine_single_char(y_list)
        if '' in x_list:
            x_list.remove('')
        if '' in y_list:
            y_list.remove('')
        
        for i, w in enumerate(x_list):
            x_list[i] = w
            
            if x_list[i] in rep_dic:
                x_list[i] = rep_dic[x_list[i]]
            
        for i, w in enumerate(y_list):
            y_list[i] = w
            
            if y_list[i] in rep_dic:
                y_list[i] = rep_dic[y_list[i]]
        if 'class' in x_list and 'class' in y_list:
            if x_list.index('class') != len(x_list) - 1 and y_list.index('class') != len(y_list) - 1:
                if x_list[x_list.index('class')+1] == y_list[y_list.index('class')+1]:
                    pre_determine.append((True,2))
                else:
                    pre_determine.append((False,2))
            else:
                pre_determine.append((None, 2))
        elif 'class' in x_list:
            if x_list.index('class') != len(x_list) - 1 and any([len(y_list[i]) == 1 for i in range(len(y_list))]):
                cl = None
                for word in y_list[::-1]:
                    if len(word) == 1:
                        cl = word
                        break
                if x_list[x_list.index('class')+1] == cl:
                    pre_determine.append((True,2))
                else:
                    pre_determine.append((False,2))
            else:
                pre_determine.append((None,1))
        elif 'class' in y_list:
            if y_list.index('class') != len(y_list) - 1 and any([len(x_list[i]) == 1 for i in range(len(x_list))]):
                cl = None
                for word in x_list[::-1]:
                    if len(word) == 1:
                        cl = word
                        break
                if y_list[y_list.index('class')+1] == cl:
                    pre_determine.append((True,2))
                else:
                    pre_determine.append((False,2))
            else:
                pre_determine.append((None,1))
        elif len(x_list[-1]) == 1 and len(y_list[-1]) == 1:
            if x_list[-1] == y_list[-1]:
                pre_determine.append((True,2))
            else:
                pre_determine.append((False,2))
        else:
            pre_determine.append((None,0))
            
            
        s_x, s_y = set(x_list), set(y_list)
        nexact_x, nexact_y = [], []
        for w in x_list:
            if w not in s_y:
                nexact_x.append(w)
        for w in y_list:
            if w not in s_x:
                nexact_y.append(w)
        dif_x, dif_y = [], []
        for w1 in nexact_x:
            abbr = False
            for w2 in nexact_y:
                abbr = is_abbr(w1, w2)
                if abbr:
                    break
            if not abbr:
                dif_x.append(sno.stem(w1))
        
        for w2 in nexact_y:
            abbr = False
            for w1 in nexact_x:
                if min(len(w1), len(w2)) == 1:
                    continue
                abbr = is_abbr(w1, w2)
                if abbr:
                    break
            if not abbr:
                dif_y.append(sno.stem(w2))
        dif_list.append((dif_x, dif_y, div))
    return pd.DataFrame(data = dif_list, columns = ['x', 'y', 'division']), pre_determine
    



In [40]:
def training_sample2(dif_df, original_df, feature_index, pre_der):
    res_list = []
    for i in range(len(dif_df)):
        feature_list = [0] * len(feature_index.keys())
        
        for w in dif_df['x'][i] + dif_df['y'][i]:
            if w in feature_index:
                feature_list[feature_index[w]] += 1
        
        
        feature_list.append(len(dif_df['x'][i] + dif_df['y'][i]) - sum(feature_list))
        feature_list.append(len(dif_df['x'][i] + dif_df['y'][i]))
        if len(dif_df['x'][i]) and 0 or len(dif_df['y'][i]) ==0:
            feature_list.append(2)
        elif len(dif_df['x'][i]) == 0 or len(dif_df['y'][i]) ==0:
            feature_list.append(1)
        else:
            feature_list.append(0)
        if pre_der[i][0]:
            feature_list.append(1)
        elif pre_der[i][0] == False:
            feature_list.append(-1)
        else:
            feature_list.append(0)
        
        
        res_list.append(feature_list)
    return pd.DataFrame(data = res_list, columns = list(feature_index.keys())+[  'Number of Unknown', 'Number of Dif', 'Blank Dif','Same Class'])




In [41]:
def get_all_gram(n, data):
    res = set()
    for i in range(len(data)):
        x, y =   data['description_x'][i],   data['description_y'][i]
        
        regex = re.compile('[^a-zA-Z]')
        x = regex.sub(' ', x)
        y = regex.sub(' ', y)
        x_list, y_list = x.split(), y.split()
        x_list, y_list = combine_single_char(x_list), combine_single_char(y_list)
        
        
        for i, w in enumerate(x_list):
            
            if x_list[i] in rep_dic:
                
                x_list[i] = rep_dic[x_list[i]]
                
            x_list[i] = sno.stem(x_list[i])
        for i, w in enumerate(y_list):
            y_list[i] = w
            
            if y_list[i] in rep_dic:
                y_list[i] = rep_dic[y_list[i]]
            y_list[i] = sno.stem(y_list[i])
        x_ngram, y_ngram = set(ngrams(x_list, n)), set(ngrams(y_list, n))
        res.update(x_ngram.difference(y_ngram))
    return sorted(list(res))

In [42]:
def training_sample_ngram(data, gram_features, n):
    ## complement n gram feature
    res_list = []
    for i in range(len(data)):
        feature_list = [0] * len(gram_features)
        x, y =   data['description_x'][i],   data['description_y'][i]
        regex = re.compile('[^a-zA-Z]')
        x = regex.sub(' ', x)
        y = regex.sub(' ', y)
        x_list, y_list = x.split(), y.split()
        x_list, y_list = combine_single_char(x_list), combine_single_char(y_list)
        
        
        for i, w in enumerate(x_list):
            
            if x_list[i] in rep_dic:
                
                x_list[i] = rep_dic[x_list[i]]
                
            x_list[i] = sno.stem(x_list[i])
        for i, w in enumerate(y_list):
            y_list[i] = w
            
            if y_list[i] in rep_dic:
                y_list[i] = rep_dic[y_list[i]]
            y_list[i] = sno.stem(y_list[i])
        x_ngram, y_ngram = set(ngrams(x_list, n)), set(ngrams(y_list, n))
        for gram in x_ngram.difference(y_ngram):
            if gram in gram_features:
                feature_list[gram_features.index(gram)] += 1
        res_list.append(feature_list)
    return pd.DataFrame(data = res_list, columns = gram_features)



In [43]:
# the best threshold is 1 after test

In [49]:
n_e = 100
fold = 5
threshold = 1
ratio = 0.8
index = [i for i in range(len(dif_df))]
accs = []
for r in range(1):
    
    random.shuffle(index)
    for f in range(fold):
        #n-fold test
        train_index, valid_index = set(index[:int(len(dif_df)*ratio*(f/fold))] + index[int(len(dif_df)*ratio*((f+1)/fold)):]) , set(index[int(len(dif_df)*ratio*(f/fold)): int(len(dif_df)*ratio*((f+1)/fold))])
        dif_df, pre_determine = difference2(training_set)
        test_dif_df, test_pre_determine = difference2(test_set)
        dif_df['label'] = training_set['same_security']
        train_dif_df = dif_df[[True if i in train_index else False for i in range(len(index))]]
        valid_dif_df = dif_df[[True if i not in train_index else False for i in range(len(index))]]
        train_ori_set = training_set[[True if i in train_index else False for i in range(len(index))]]
        valid_ori_set = training_set[[True if i not in train_index else False for i in range(len(index))]]
        valid_pre = dif_df[[True if i not in train_index else False for i in range(len(index))]]
        train_dif_df.reset_index(inplace = True)
        valid_dif_df.reset_index(inplace = True)
        train_ori_set.reset_index(inplace = True)
        valid_ori_set.reset_index(inplace = True)
        test_dif_df.reset_index(inplace = True)
        
        s2 = dict()
        for i in range(len(train_dif_df)):
            for w in train_dif_df['x'][i] + train_dif_df['y'][i]:

                s2[w] = s2.get(w, 0) + 1
        
        features2 = sorted([ k for k, v in s2.items() if v >threshold ])
        feature_index2 = {features2[i] : i for i in range(len(features2))}
        
        train_pre_der = [pre_determine[i]  for i in range(len(index)) if i in train_index]
        valid_pre_der = [pre_determine[i]  for i in range(len(index)) if i in valid_index]

        X, y = training_sample2(train_dif_df,train_ori_set, feature_index2, train_pre_der), train_dif_df['label']
        valid_X, valid_y = training_sample2(valid_dif_df, valid_ori_set, feature_index2, valid_pre_der), valid_dif_df['label']
        et = ET(n_estimators = n_e, warm_start =  True, n_jobs = -1)
        et.fit(X, y)
        prediction = et.predict(valid_X)
        for di_i in range(len(valid_dif_df['division'])):
            if valid_dif_df['division'][di_i] == True:
                prediction[di_i] = True
            elif valid_dif_df['division'][di_i] == False:
                prediction[di_i] = False


        res = prediction == valid_y
        pre, recall, f1, sth = prfs(valid_y, prediction, average = 'binary')
        acc = res.value_counts()[True]/len(res)
        accs.append(acc)

print(np.average(accs), np.std(accs))

0.9439653555657852 0.008491164442243328


In [57]:
X

,x,y,division,label
0,[],[index],None,True
1,[],"[strateg, tr, fundament]",None,True
2,[admin],"[fund, inst]",None,False
3,[],[c],None,True
4,"[class, a]",[],None,True
5,"[new, div]",[],True,True
6,[],[new],None,True
7,[],[],None,True
8,"[strateg, tr]","[cap, etf]",None,True
9,[],[],None,True


In [55]:
fold = 5
threshold = 1
ratio = 0.8
index = [i for i in range(len(dif_df))]
random.shuffle(index)
accs = []
for i in range(1):
        for r in range(1):
            for f in range(fold):
                #n-fold test
                train_index, valid_index = set(index[:int(len(dif_df)*ratio*(f/fold))] + index[int(len(dif_df)*ratio*((f+1)/fold)):]) , set(index[int(len(dif_df)*ratio*(f/fold)): int(len(dif_df)*ratio*((f+1)/fold))])
                dif_df, pre_determine = difference2(training_set)
                test_dif_df, test_pre_determine = difference2(test_set)
                dif_df['label'] = training_set['same_security']
                train_dif_df = dif_df[[True if i in train_index else False for i in range(len(index))]]
                valid_dif_df = dif_df[[True if i not in train_index else False for i in range(len(index))]]
                train_ori_set = training_set[[True if i in train_index else False for i in range(len(index))]]
                valid_ori_set = training_set[[True if i not in train_index else False for i in range(len(index))]]
                valid_pre = dif_df[[True if i not in train_index else False for i in range(len(index))]]
                train_dif_df.reset_index(inplace = True)
                valid_dif_df.reset_index(inplace = True)
                train_ori_set.reset_index(inplace = True)
                valid_ori_set.reset_index(inplace = True)
                test_dif_df.reset_index(inplace = True)
                s2 = dict()
                for i in range(len(train_dif_df)):
                    for w in train_dif_df['x'][i] + train_dif_df['y'][i]:
                        s2[w] = s2.get(w, 0) + 1
                features2 = sorted([ k for k, v in s2.items() if v >threshold ])
                feature_index2 = {features2[i] : i for i in range(len(features2))}
                train_pre_der = [pre_determine[i]  for i in range(len(index)) if i in train_index]
                valid_pre_der = [pre_determine[i]  for i in range(len(index)) if i in valid_index]
                X, y = training_sample2(train_dif_df,train_ori_set, feature_index2, train_pre_der), train_dif_df['label']
                valid_X, valid_y = training_sample2(valid_dif_df, valid_ori_set, feature_index2, valid_pre_der), valid_dif_df['label']
                two_gram_features = get_all_gram(2, train_ori_set)
                train_2gram_table = training_sample_ngram(train_ori_set, two_gram_features,2)
                valid_2gram_table = training_sample_ngram(valid_ori_set, two_gram_features,2)
                X = pd.concat([X, train_2gram_table], axis=1)
                valid_X = pd.concat([valid_X, valid_2gram_table], axis=1)
                mlp = MLPClassifier(solver='adam', alpha=2, hidden_layer_sizes=tuple((200,)*3), tol =0.0001, verbose =False, learning_rate = 'adaptive' )
                mlp.fit(X, y)
                prediction = mlp.predict(valid_X)
                for di_i in range(len(valid_dif_df['division'])):
                    if valid_dif_df['division'][di_i] == True:
                        prediction[di_i] = True
                    elif valid_dif_df['division'][di_i] == False:
                        prediction[di_i] = False
                res = prediction == valid_y
                pre, recall, f1, sth = prfs(valid_y, prediction, average = 'binary')
                acc = res.value_counts()[True]/len(res)
                accs.append(acc)

print('acc:', np.average(accs), 'std:', np.std(accs))

acc: 0.9550525974800947 std: 0.008361069940720983


In [54]:
X.columns

Index([                  'a',                'aapl',                  'ad',
                     'admin',           'administr',               'admir',
                    'admira',                 'adr',                 'adv',
                     'advtg',
       ...
           ('yield', 'muni'),  ('yield', 'municip'),  ('yieldbond', 'etf'),
               ('yld', 'bd'), ('york', 'communiti'),    ('york', 'mellon'),
           ('yr', 'treasri'),      ('yr', 'treasu'),    ('yr', 'treasuri'),
         ('zillow', 'group')],
      dtype='object', length=2109)

In [77]:
fold = 5
threshold = 1
ratio = 0.8
index = [i for i in range(len(dif_df))]
random.shuffle(index)
neuron, layer = 200, 3
accs = []
for r in range(5):
    for f in range(fold):
        #n-fold test

        train_index, valid_index = set(index[:int(len(dif_df)*ratio*(f/fold))] + index[int(len(dif_df)*ratio*((f+1)/fold)):]) , set(index[int(len(dif_df)*ratio*(f/fold)): int(len(dif_df)*ratio*((f+1)/fold))])
        dif_df, pre_determine = difference2(training_set)
        test_dif_df, test_pre_determine = difference2(test_set)
        dif_df['label'] = training_set['same_security']
        train_dif_df = dif_df[[True if i in train_index else False for i in range(len(index))]]
        valid_dif_df = dif_df[[True if i not in train_index else False for i in range(len(index))]]
        train_ori_set = training_set[[True if i in train_index else False for i in range(len(index))]]
        valid_ori_set = training_set[[True if i not in train_index else False for i in range(len(index))]]
        valid_pre = dif_df[[True if i not in train_index else False for i in range(len(index))]]
        train_dif_df.reset_index(inplace = True)
        valid_dif_df.reset_index(inplace = True)
        train_ori_set.reset_index(inplace = True)
        valid_ori_set.reset_index(inplace = True)
        test_dif_df.reset_index(inplace = True)

        s2 = dict()
        for i in range(len(train_dif_df)):
            for w in train_dif_df['x'][i] + train_dif_df['y'][i]:

                s2[w] = s2.get(w, 0) + 1

        features2 = sorted([ k for k, v in s2.items() if v >threshold ])
        feature_index2 = {features2[i] : i for i in range(len(features2))}

        train_pre_der = [pre_determine[i]  for i in range(len(index)) if i in train_index]
        valid_pre_der = [pre_determine[i]  for i in range(len(index)) if i in valid_index]

        X, y = training_sample2(train_dif_df,train_ori_set, feature_index2, train_pre_der), train_dif_df['label']
        valid_X, valid_y = training_sample2(valid_dif_df, valid_ori_set, feature_index2, valid_pre_der), valid_dif_df['label']
        two_gram_features = get_all_gram(2, train_ori_set)
        train_2gram_table = training_sample_ngram(train_ori_set, two_gram_features,2)
        valid_2gram_table = training_sample_ngram(valid_ori_set, two_gram_features,2)

        X = pd.concat([X, train_2gram_table], axis=1)
        valid_X = pd.concat([valid_X, valid_2gram_table], axis=1)
        mlp = MLPClassifier(solver='adam', alpha=1, hidden_layer_sizes=tuple((200,)*3), tol =0.0001, verbose =False, learning_rate = 'adaptive' )
        mlp.fit(X, y)
        prediction = mlp.predict(valid_X)
        for di_i in range(len(valid_dif_df['division'])):
            if valid_dif_df['division'][di_i] == True:
                prediction[di_i] = True
            elif valid_dif_df['division'][di_i] == False:
                prediction[di_i] = False


        res = prediction == valid_y
        pre, recall, f1, sth = prfs(valid_y, prediction, average = 'binary')
        acc = res.value_counts()[True]/len(res)
        accs.append(acc)

print('neuron', neuron,'layer', layer, 'acc:', np.average(accs), 'std:', np.std(accs))

neuron 1000 layer 1 acc: 0.9569195096585001 std: 0.009006273865714058


In [55]:
len(X.columns)

2096

In [44]:
def score_func(true_y, prediction ):
    res = prediction == true_y
    acc = res.value_counts()[True]/len(res)
    return acc

In [45]:
from sklearn.metrics import make_scorer

In [46]:
score = make_scorer(score_func, greater_is_better=True)


In [49]:
def mlp_param_selection(X, y, nfolds):
    alphas = [0.01, 0.1, 0.5, 1, 1.5, 2, 5]
    max_iter = [200, 300, 500, 1000, 2000]
    layer, nodes = [2,3,4,5,6,7,8,9,10],[100,200,300,500,700,1000,2000]
    hidden_layer_sizes = [(layer[i], nodes[j]) for i in range(len(layer)) for j in range(len(nodes))]
    param_grid = {'alpha': alphas, 'hidden_layer_sizes' : hidden_layer_sizes, 'max_iter':max_iter}
    grid_search = GridSearchCV(MLPClassifier(learning_rate = 'adaptive'), param_grid, cv=nfolds, verbose = 10, scoring = score, n_jobs = 6)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

In [87]:
index = [i for i in range(len(dif_df))]

threshold = 1
f= 0
fold = 5
train_index, valid_index = set(index[:int(len(dif_df)*ratio*(f/fold))] + index[int(len(dif_df)*ratio*((f+1)/fold)):]) , set(index[int(len(dif_df)*ratio*(f/fold)): int(len(dif_df)*ratio*((f+1)/fold))])
dif_df, pre_determine = difference2(training_set)
test_dif_df, test_pre_der = difference2(test_set)
dif_df['label'] = training_set['same_security']
train_dif_df = dif_df[[True if i in train_index else False for i in range(len(index))]]
valid_dif_df = dif_df[[True if i not in train_index else False for i in range(len(index))]]
train_ori_set = training_set[[True if i in train_index else False for i in range(len(index))]]
valid_ori_set = training_set[[True if i not in train_index else False for i in range(len(index))]]
valid_pre = dif_df[[True if i not in train_index else False for i in range(len(index))]]
train_dif_df.reset_index(inplace = True)
valid_dif_df.reset_index(inplace = True)
train_ori_set.reset_index(inplace = True)
valid_ori_set.reset_index(inplace = True)
test_dif_df.reset_index(inplace = True)
s2 = dict()
for i in range(len(train_dif_df)):
    for w in train_dif_df['x'][i] + train_dif_df['y'][i]:

        s2[w] = s2.get(w, 0) + 1
train_pre_der = [pre_determine[i]  for i in range(len(index)) if i in train_index]
valid_pre_der = [pre_determine[i]  for i in range(len(index)) if i in valid_index]
features2 = sorted([ k for k, v in s2.items() if v >threshold ])
feature_index2 = {features2[i] : i for i in range(len(features2))}
X, y = training_sample2(train_dif_df,train_ori_set, feature_index2, train_pre_der), train_dif_df['label']
valid_X, valid_y = training_sample2(valid_dif_df, valid_ori_set, feature_index2, valid_pre_der), valid_dif_df['label']

test_X = training_sample2(test_dif_df, test_set, feature_index2, test_pre_der)

two_gram_features = get_all_gram(2, train_ori_set)
train_2gram_table = training_sample_ngram(train_ori_set, two_gram_features,2)
valid_2gram_table = training_sample_ngram(valid_ori_set, two_gram_features,2)

test_2gram_table = training_sample_ngram(test_set, two_gram_features,2)

X = pd.concat([X, train_2gram_table], axis=1)

valid_X = pd.concat([valid_X, valid_2gram_table], axis=1)
train_X = pd.concat([X, valid_X])
train_X.reset_index(inplace= True)
train_X = train_X.drop('level_0', axis = 1)
train_y = y.append(valid_y, ignore_index = True)

test_X = pd.concat([test_X, test_2gram_table], axis=1)

In [104]:
list(test_dif_df['division']).count(True)

38

In [96]:
mlp = MLPClassifier(solver='adam', alpha=1, hidden_layer_sizes=tuple((200,)*3), tol =0.0001, verbose =False, learning_rate = 'adaptive' )
mlp.fit(X, y)
prediction = mlp.predict(test_X)
for di_i in range(len(test_dif_df['division'])):
    if test_dif_df['division'][di_i] == True:
        prediction[di_i] = True
    elif test_dif_df['division'][di_i] == False:
        prediction[di_i] = False



In [98]:
test_set['same_security'] = prediction
test_set.to_csv(path_or_buf = '/Users/accy/Documents/GitHub/Vanguard_Kaggle/submission/Phase 1 - submission/submission_phase1_team1.csv', index = False)

In [108]:
difference2(test_set)

(                                       x                              y  \
 0                                     []                             []   
 1                                     []                            [a]   
 2                                     []                             []   
 3                                [admin]                       [market]   
 4                                 [fund]                             []   
 5                                  [div]                          [com]   
 6                       [ex, fid, class]                             []   
 7                                [admin]        [fund, investor, share]   
 8    [de, exportacio, class, stock, npv]                             []   
 9                                  [rep]   [spon, ad, repr, cls, a, us]   
 10                                    []                     [inc, div]   
 11                            [inc, com]                             []   
 12         